In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

## **Data preparation and import**

In [ ]:
test = pd.read_csv("/content/Test.csv")
train = pd.read_csv("/content/Train.csv")
test = test.drop(['Trip'],axis=1)
train = train.drop(['Trip'],axis=1)

In [ ]:
len_test = len(test)
len_train = len(train)
total = len_test + len_train
print("Training data has {} features, Test data has {} features. Train-test data ratio is {}:{}".format(len_train,len_test,round((len_train/total)*100),round((len_test/total)*100)))

Training data has 107085 features, Test data has 60172 features. Train-test data ratio is 64:36


In [ ]:
data = pd.concat([train,test], axis=0)
data.isnull().sum()

Trip                  0
VehicleSpeed_km_h_    0
Battery_Status        0
dtype: int64

**Splitting both train and test data into respective input and target features (X,y)**

In [ ]:
y_train = train['Battery_Status']
X_train = train.drop(['Battery_Status'],axis=1)
y_test = test['Battery_Status']
X_test = test.drop(['Battery_Status'],axis=1)

In [ ]:
norm_scaler = MinMaxScaler()
X_train = norm_scaler.fit_transform(X_train)
X_test = norm_scaler.fit_transform(X_test)

In [ ]:
from collections import Counter
Counter(y_train)
Counter(y_test)

Counter({0: 44687, 1: 15485})

## **Model Building**

**We will implement KNN and Random Forest (Ensemble of Trees) for this task. SVM was considered, however, it takes too much time to train an SVM.**

In [ ]:
KNN_params = [{"n_neighbors":5}, {"n_neighbors":7}, {"n_neighbors":15}, {"n_neighbors":19}, {"n_neighbors":round(np.sqrt(len(X_train)))}]
RF_params = [{"criterion": "gini"}, {"criterion": "entropy"}]

In [ ]:
modelclasses = [
    ["KNN", KNeighborsClassifier, KNN_params],
    ["EL", RandomForestClassifier, RF_params]
]

**The next cell will take a few minutes to run**

In [ ]:
results = []
for modelname, Model, params_list in modelclasses:
    for params in params_list:

        # model performance dictionary
        score = {'accuracy':0,'recall':0,'precision':0,'f1':0,'auroc':0,'conf_matrix':0}
        
        # obtain hyperparameters defined prior
        model = Model(**params)
        
        # fit model to train data
        model.fit(X_train, y_train) 
        
        # obtain predictions for test set
        predictions = model.predict(X_test)

        # obtain metrics and add to dictionary
        score['accuracy'] = metrics.accuracy_score(y_test, predictions)
        score['recall'] = metrics.recall_score(y_test, predictions)
        score['precision'] = metrics.precision_score(y_test, predictions)
        score['f1'] = metrics.f1_score(y_test, predictions)
        score['auroc'] = metrics.roc_auc_score(y_test, predictions)
        score['conf_matrix'] = metrics.confusion_matrix(y_test, predictions)
        
        # append results to list
        results.append((modelname, model, params, score))

In [ ]:
results

## **Models Evaluation**

In [ ]:
for i in results:
  if 'KNN' in i:
    # display K value
    print(i[2])
    
    # display metrics
    print("Accuracy: {}".format(i[3]['accuracy']))
    print("Recall: {}".format(i[3]['recall']))
    print("Precision: {}".format(i[3]['precision']))
    print("F1 score: {}".format(i[3]['f1']))
    print("AUROC: {}".format(i[3]['auroc']))
    
    # plot conf matrix
    fig, ax = plt.subplots(figsize=(7.5, 7.5))
    ax.matshow(score['conf_matrix'], cmap=plt.cm.Blues, alpha=0.3)
    for i in range(score['conf_matrix'].shape[0]):
        for j in range(score['conf_matrix'].shape[1]):
            ax.text(x=j, y=i,s=score['conf_matrix'][i, j], va='center', ha='center', size='xx-large')
    
    plt.xlabel('Predictions', fontsize=18)
    plt.ylabel('Actuals', fontsize=18)
    plt.title('Confusion Matrix', fontsize=18)
    plt.show()

In [ ]:
for i in results:
  if 'EL' in i:
    # display criterion value
    print(i[2])
    
    # display metrics
    print("Accuracy: {}".format(i[3]['accuracy']))
    print("Recall: {}".format(i[3]['recall']))
    print("Precision: {}".format(i[3]['precision']))
    print("F1 score: {}".format(i[3]['f1']))
    print("AUROC: {}".format(i[3]['auroc']))
    
    # plot conf matrix
    fig, ax = plt.subplots(figsize=(7.5, 7.5))
    ax.matshow(score['conf_matrix'], cmap=plt.cm.Blues, alpha=0.3)
    for i in range(score['conf_matrix'].shape[0]):
        for j in range(score['conf_matrix'].shape[1]):
            ax.text(x=j, y=i,s=score['conf_matrix'][i, j], va='center', ha='center', size='xx-large')
    
    plt.xlabel('Predictions', fontsize=18)
    plt.ylabel('Actuals', fontsize=18)
    plt.title('Confusion Matrix', fontsize=18)
    plt.show()